In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from tqdm import tqdm
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import time
import torch
plt.rcParams['figure.figsize'] = (8, 8)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sondre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sondre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\Sondre\Documents\kodegit\emojify\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/train.txt', sep=";", names=['text', 'label'])
df

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [3]:
print(df.text[1599])
print(len(df))

i just feel so fucked up these days
16000


In [4]:
#lage list of sentences
list_of_sentences = []
for i in range(len(df)):
    list_of_sentences.append(df.text[i])

#lage list of word
list_of_word = []
for i in range(len(list_of_sentences)):
    list_of_word.extend(word_tokenize(list_of_sentences[i]))
    

#lage bag of words
bag_of_words = {}
for word in list_of_word:
    if word in bag_of_words.keys():
        bag_of_words[word] += 1
    else:
        bag_of_words[word] = 1



#før fjerning
print("Length before removing stopwords:", len(bag_of_words.keys()))
print("Number of 'I':", bag_of_words['i'])

#definere stopwords
stop_words_english = set(stopwords.words('english'))
bag_of_words_keys = list(bag_of_words.keys())


#fjerne stopwords
for i in bag_of_words_keys:
    if i in stop_words_english:
        bag_of_words.pop(i)
        

#sjekke om de er fjerna
print("Length after removing stopwords:", len(bag_of_words.keys()))

Length before removing stopwords: 15210
Number of 'I': 25859
Length after removing stopwords: 15062


In [5]:
#få en liste fra keys
def getList(dict):
    list = []
    for value in dict.values():
        list.append(value)
         
    return list

#lage bag of words som gir ut antall av hvert eneste ord til en enkelt setning
def make_bag_of_words(counter, setning):
    bag_of_word = {}
    list_setning = word_tokenize(setning)
    for word in bag_of_words.keys():
        bag_of_word[word] = 0
    
    for word in bag_of_words.keys():
        if word in list_setning:
            for ord in list_setning:
                if word == ord:
                    bag_of_word[word] += 1
        else:
            bag_of_word[word] = 0
    
    list_bag_of_word = getList(bag_of_word)
    list_bag_of_word.insert(0, counter)
    return list_bag_of_word

g = (make_bag_of_words(89, "i i am feeling feeling grouchy"))
#print(g)

In [6]:
#øverste rad er orda
bow_matrise = []
bow_matrise.append(getList(bag_of_words))

#legge inn antall ordforekomst i hver setning
iteration_counter = 1
for setence in tqdm(list_of_sentences):
    bow_matrise.append(make_bag_of_words(iteration_counter ,setence))
    iteration_counter += 1



100%|██████████| 16000/16000 [01:31<00:00, 174.43it/s]


In [7]:
#test for å se hvilkene ord setningene inneholder
number = 15936

liste = (bow_matrise[number])
for tall in range(len(liste)):
    if liste[tall] != 0:
        print("word index:", tall, ", antall: ", liste[tall])
print (list_of_sentences[number])

word index: 0 , antall:  15936
word index: 2 , antall:  1
word index: 772 , antall:  1
word index: 940 , antall:  1
word index: 3338 , antall:  1
im better than the rest of you feeling but a feeling of being accepted


In [8]:
#funksjon for å fjerne øverste rad og første kolonne
def fjern_f(matrise):
    del matrise[0]
    for i in range(len(matrise)):
        del matrise[i][0]
    return matrise

In [9]:
#values til ordlista:
kolonner = fjern_f(bow_matrise)

#keys til ordlista:
kolonne_navn = bag_of_words_keys

#ordlista med ord som keys: og tilhørende value som en liste med antall av hvert ord
dict_til_def = {kolonne_navn[i]: kolonner[i] for i in tqdm(range(len(kolonne_navn)))}



100%|██████████| 15210/15210 [00:00<00:00, 3041060.34it/s]


In [10]:
#lage dataframe
df_ai = pd.DataFrame(data=dict_til_def)

In [11]:
#sjekke den nye dataframen
df_ai

,i,didnt,feel,humiliated,can,go,from,feeling,so,hopeless,...,pandora,cosmopolitian,monkees,tearing,celebrities,irrelevant,braeden,calvin,beanbag,subbing
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,1,1,1,1,...,0,0,0,1,0,1,1,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,1,1,1,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15057,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15060,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
feelings = df
feelings.head()

###alt under er hentet fra nettet

X = feelings['text']
Y = feelings['label']

In [13]:
from sklearn.model_selection import train_test_split


#teste greia
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [14]:
vectorizer = CountVectorizer(lowercase=True, stop_words='english')

# fit and transform X_train
X_train_bow = vectorizer.fit_transform(X_train)

# Transform X_test
X_test_bow = vectorizer.transform(X_test)

# Print shape of X_train_bow and X_test_bow
print(X_train_bow.shape)
print(X_test_bow.shape)

(12000, 12770)
(4000, 12770)


In [16]:
from sklearn.naive_bayes import MultinomialNB


# Create a MultinomialNB object
clf = MultinomialNB()

# Fit the classifier
clf.fit(X_train_bow, y_train)

# Measure the accuracy
accuracy = clf.score(X_test_bow, y_test)
print("Nøyaktigheten til classifieren basert på dette settet er:", accuracy*100, "%")

# Predict the feeling
review = 'i am angry at you'
print("Setning: ", review)
prediction = clf.predict(vectorizer.transform([review]))[0]
print("Passende følelse til denne setningen:" , (prediction))

Nøyaktigheten til classifieren basert på dette settet er: 77.525 %
Setning:  i am angry at you
Passende følelse til denne setningen: anger
